In [1]:
from collections import deque

In [2]:
filename = "sample.txt"
# filename = "input.txt"
with open(filename, encoding="utf-8") as f:
    data = f.read()

lines = data.strip().split("\n")

https://adventofcode.com/2024/day/18

In [3]:
# Parse lines into {pos: line number}
blocks = dict()
for i, line in enumerate(lines):
    x, y = map(int, line.split(","))
    blocks[complex(x, y)] = i

In [4]:
## Part 1
# Pathfind from 0,0 to 70,70 after 1024 blocks have fallen
# For the sample, grid ranges to 6,6 instead, and only 12 bytes fall
n_blocks = 12
x_hi, y_hi = 6, 6
# n_blocks = 1024
# x_hi, y_hi = 70, 70

p1_blocks = {k: v for (k, v) in blocks.items() if v < n_blocks}
start_pos = complex(0, 0)
end_pos = complex(x_hi, y_hi)

In [5]:
def in_bounds(pos):
    return (0 <= pos.real <= x_hi) and (0 <= pos.imag <= y_hi)

def adjacent(pos):
    return [pos + step for step in (1, 1j, -1, -1j)]

def get_path(parents: dict, end: complex):
    out = [end]
    pos = parents.get(end)
    while pos is not None:
        out.append(pos)
        pos = parents.get(pos)
    return out[::-1]

def bfs(blocks, start, end):
    # Based on pseudocode from wiki on breadth-first search
    # seen = {start}
    parent = {start: None}
    frontier = deque([start])
    while frontier:
        # Note: since all steps are the same size, BFS is guaranteed to find the shortest path if we explore frontier FIFO
        pos = frontier.popleft()  
        if pos == end:
            return get_path(parent, pos)
        # Mark all reachable neighbours of pos as seen
        for n in adjacent(pos):
            if n in parent:
                # Already seen
                continue
            if (not in_bounds(n)) or (n in blocks):
                # Blocked
                continue
            parent[n] = pos
            frontier.append(n)
    print(f"BFS couldn't find path from {start} to {end}. {parent=}")


In [6]:
result = bfs(p1_blocks, start_pos, end_pos)
len(result) - 1  # Start doesn't cost a step

22

In [7]:
## Part 2
# What are the coordinates of the first byte that will prevent the exit from being reachable from the start?
# Opt 1: Run BFS to find a route. Add blocks until one hits route, then recalculate BFS. Repeat until a path can't be found
# Opt 1a: Similar to above, but in a binary-search manner to run BFS fewer times
# Opt 2: Something with graph-related? Start with a fully-connected graph, remove edges until start and goal aren't in the same sub-graph
def binary_search_f(f, vs):
    # Binary search: find the first value of ls where predicate f is True
    # From https://stackoverflow.com/a/42119794
    lo = 0
    hi = len(vs)
    while lo < hi:
        mid = (lo + hi) // 2
        if f(vs[mid]):
            hi = mid
        else:
            lo = mid + 1
    return lo

def reachable(all_blocks, start, end, n_blocks):
    # Note: blocks implementation could be cleaner by taking set([block positions][:n])
    print(f"Running bfs for {n_blocks=}")
    blocks = {k: v for k, v in all_blocks.items() if v < n_blocks}
    path = bfs(blocks, start, end)
    return path is None

In [8]:
result_index = binary_search_f(lambda n: reachable(blocks, start_pos, end_pos, n), list(range(len(blocks) + 1)))
result_index

Running bfs for n_blocks=13
Running bfs for n_blocks=20
Running bfs for n_blocks=23
BFS couldn't find path from 0j to (6+6j). parent={0j: None, 1j: 0j, 2j: 1j, 3j: 2j, (1+3j): 3j, (2+3j): (1+3j), (2+2j): (2+3j), (3+2j): (2+2j), (3+1j): (3+2j), (4+1j): (3+1j), (4+0j): (4+1j), (5+0j): (4+0j), (6+0j): (5+0j)}
Running bfs for n_blocks=22
BFS couldn't find path from 0j to (6+6j). parent={0j: None, 1j: 0j, 2j: 1j, 3j: 2j, (1+3j): 3j, (2+3j): (1+3j), (2+2j): (2+3j), (3+2j): (2+2j), (3+1j): (3+2j), (4+1j): (3+1j), (4+0j): (4+1j), (5+0j): (4+0j), (6+0j): (5+0j)}
Running bfs for n_blocks=21
BFS couldn't find path from 0j to (6+6j). parent={0j: None, (1+0j): 0j, 1j: 0j, (2+0j): (1+0j), 2j: 1j, 3j: 2j, (1+3j): 3j, (2+3j): (1+3j), (2+2j): (2+3j), (3+2j): (2+2j), (3+1j): (3+2j), (4+1j): (3+1j), (4+0j): (4+1j), (5+0j): (4+0j), (6+0j): (5+0j)}


21

In [9]:
lines[result_index - 1]

'6,1'